In [3]:
import numpy as np
import scipy.io as sio
import matplotlib.pyplot as plt
from skimage.metrics import normalized_root_mse
from skimage.draw import line
import mat73


In [10]:
data = mat73.loadmat("data/assignmentMathImagingRecon_chestCT.mat")
data

{'imageAC': array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]], shape=(512, 512))}

In [11]:
chestCT = data["imageAC"]
image_size = chestCT.shape[0]

In [5]:
# Define function to generate Radon transform matrix
def radon_matrix(size, angles):
    """
    Construct the system matrix A manually for ART.
    """
    A = []
    n = size  # Assume square image (n x n)
    center = n // 2
    for theta in angles:
        theta_rad = np.deg2rad(theta)
        sin_theta, cos_theta = np.sin(theta_rad), np.cos(theta_rad)

        for x in range(n):
            y = int(center + (x - center) * sin_theta)
            x_new = int(center + (x - center) * cos_theta)
            if 0 <= x_new < n and 0 <= y < n:
                row = np.zeros((n, n))
                rr, cc = line(center, center, x_new, y)
                row[rr, cc] = 1
                A.append(row.flatten())

    return np.array(A)

In [7]:
# Define function for ART algorithm
def myART(projections, angles, lambda_val, max_iter=50, ordering='sequential'):
    """
    ART implementation with different orderings.
    projections: measured sinogram (1D array)
    angles: angles used in measurement
    lambda_val: step size
    max_iter: number of iterations
    ordering: 'sequential', 'random', or 'cyclic'
    """
    n = image_size
    A = radon_matrix(n, angles)
    x = np.zeros(n * n)  # Initial estimate

    num_projections = len(angles)
    if ordering == 'random':
        order = np.random.permutation(num_projections)
    elif ordering == 'cyclic':
        order = np.tile(np.arange(num_projections), 2)[:num_projections]
    else:
        order = np.arange(num_projections)

    rrmse_list = []

    for i in range(max_iter):
        for j in order:
            row = A[j]
            projection_diff = projections[j] - np.dot(row, x)
            x += (lambda_val / np.linalg.norm(row) ** 2) * projection_diff * row

        # Compute RRMSE
        reconstructed = x.reshape(n, n)
        rrmse = normalized_root_mse(chestCT, reconstructed)
        rrmse_list.append(rrmse)

    return x.reshape(n, n), rrmse_list

In [12]:
# Simulate Radon transform & add noise
angles = np.arange(0, 180, 1)  # 180 angles spanning 180 degrees
A = radon_matrix(image_size, angles)
sinogram = A @ chestCT.flatten()

: 

In [ ]:
# Add Gaussian noise (5% of intensity range)
noise_std = 0.05 * (np.max(sinogram) - np.min(sinogram))
sinogram_noisy = sinogram + np.random.normal(0, noise_std, sinogram.shape)